In [1]:
import json
import datetime
from enum import Enum, auto

from bs4 import BeautifulSoup
import requests
import functools

In [2]:
class Journal(Enum):
    EL_PAIS = auto()
    ABC = auto()

In [3]:
JOURNAL = Journal.ABC

In [4]:
if JOURNAL == Journal.EL_PAIS:
    url = "https://elpais.com/espana/madrid/"
else:
    url = "https://www.abc.es/espana/madrid/?ref=https%3A%2F%2Fwww.abc.es%2Fespana%2Fmadrid%2F"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    html_content = response.text
else:
    print("Error: Unable to fetch the webpage.")

In [5]:
soup = BeautifulSoup(html_content, "lxml")

In [6]:
@functools.cache
def get_description(link_noticia: str):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
    }
    response = requests.get(link_noticia, headers=headers)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, "lxml")
        description = soup.find("h2")
        return description.text.strip()
    else:
        return "Error: Unable to fetch the webpage."

In [7]:
def get_date_from_link(link_noticia: str, journal: Journal):
    if journal == Journal.EL_PAIS:
        # Example link:
        # https://elpais.com/espana/madrid/2024-04-02/nombre-noticia.html
        date_str = link_noticia.split("/")[-2]
        # Format to YYYY-MM-DD
        date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    else:
        # Example link:
        # https://www.abc.es/espana/madrid/nombre-noticia-20240402180803-nt.html
        last_part = link_noticia.split("/")[-1]
        date_str_and_code = last_part.split("-")[-2]
        date_str = date_str_and_code[:8]
        # format to YYYY-MM-DD
        date = datetime.datetime.strptime(date_str, "%Y%m%d")
        date_str = date.strftime("%Y-%m-%d")
        return date_str

In [8]:
titles = soup.find_all("h2")

data = []
today = datetime.datetime.now().strftime("%Y-%m-%d")
for title in titles:
    a = title.find("a")
    if a is None:
        continue
    link = a["href"]
    date_from_link = get_date_from_link(link, JOURNAL)
    if date_from_link != today:
        continue
    noticia = {}
    noticia["title"] = title.text.strip()
    noticia["link"] = link
    noticia["description"] = get_description(noticia["link"])
    data.append(noticia)
data

[{'title': 'Quince heridos, dos graves, tras chocar un autobús contra un muro en Valdemoro, Madrid',
  'link': 'https://www.abc.es/espana/madrid/quince-heridos-dos-graves-tras-chocar-autobus-20240403100448-nt.html',
  'description': 'El siniestro se ha producido en torno a las 7.20 horas de este miércoles'},
 {'title': 'El Palacio de Madrid similar al de Versalles que casi nadie conoce y se puede visitar: donde está y cómo llegar',
  'link': 'https://www.abc.es/espana/madrid/palacio-madrid-similar-versalles-conoce-puede-visitar-20240403060000-nt.html',
  'description': 'Se trata de una edificación construida hace más de 200 años pero que se mantiene en excelente estado de conservación'}]

In [11]:
with open(
    f"noticias_{datetime.datetime.now().strftime('%Y-%m-%d')}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

In [12]:
with open("noticias_2024-04-03.json", "r", encoding="utf-8") as f:
    load_data = json.load(f)

In [13]:
load_data

[{'title': 'Quince heridos, dos graves, tras chocar un autobús contra un muro en Valdemoro, Madrid',
  'link': 'https://www.abc.es/espana/madrid/quince-heridos-dos-graves-tras-chocar-autobus-20240403100448-nt.html',
  'description': 'El siniestro se ha producido en torno a las 7.20 horas de este miércoles'},
 {'title': 'El Palacio de Madrid similar al de Versalles que casi nadie conoce y se puede visitar: donde está y cómo llegar',
  'link': 'https://www.abc.es/espana/madrid/palacio-madrid-similar-versalles-conoce-puede-visitar-20240403060000-nt.html',
  'description': 'Se trata de una edificación construida hace más de 200 años pero que se mantiene en excelente estado de conservación'}]